# Kafka consume

In [5]:
from confluent_kafka import Consumer, KafkaException
import json
from typing import List, Dict

CONFIG = {
    "bootstrap.servers": "localhost:9092",
    "group.id": "notebook-consumer-group",
    "auto.offset.reset": "earliest",
}


def consume_kafka_messages(topic: str, max_messages: int) -> List[Dict]:
    consumer = Consumer(CONFIG)
    consumer.subscribe([topic])

    messages = []
    print(f"Subscribed to topic: {topic}")
    print(f"Waiting for up to {max_messages} messages...")

    try:
        while len(messages) < max_messages:
            msg = consumer.poll(timeout=2.0)

            if msg is None:
                print("No message received.")
                continue
            if msg.error():
                raise KafkaException(msg.error())

            try:
                value = json.loads(msg.value().decode("utf-8"))
                messages.append(value)
                print(f"Received message {len(messages)} of {max_messages}")
            except json.JSONDecodeError:
                print("Received non-JSON message, skipping.")

    finally:
        consumer.close()

    return messages


messages = consume_kafka_messages("storefront.public.orders", 5)
for msg in messages:
    print(json.dumps(msg, indent=2))

Subscribed to topic: storefront.public.orders
Waiting for up to 5 messages...
No message received.
Received message 1 of 5
Received message 2 of 5
Received message 3 of 5
Received message 4 of 5
Received message 5 of 5
{
  "schema": {
    "type": "struct",
    "fields": [
      {
        "type": "struct",
        "fields": [
          {
            "type": "int32",
            "optional": false,
            "default": 0,
            "field": "id"
          },
          {
            "type": "int32",
            "optional": false,
            "field": "customer_id"
          },
          {
            "type": "string",
            "optional": false,
            "field": "status"
          },
          {
            "type": "bytes",
            "optional": true,
            "name": "org.apache.kafka.connect.data.Decimal",
            "version": 1,
            "parameters": {
              "scale": "2",
              "connect.decimal.precision": "10"
            },
            "field": "t